In [3]:
# Import required packages
import torch
import os
import numpy as np

from utils.solver import Solver, DEFAULT_SOLVER_PARAM_M7, DEFAULT_SOLVER_PARAM_M3
from utils.robot import get_robot, sample_P_path, sample_J_traj
from utils.utils import load_numpy, save_numpy, nearest_neighbor_F

In [4]:
robot = get_robot()
solver = Solver(robot=robot, solver_param=DEFAULT_SOLVER_PARAM_M7)

WorldModel::LoadRobot: /home/luca/.cache/jrl/temp_urdfs/panda_arm_hand_formatted_link_filepaths_absolute.urdf
joint mimic: no multiplier, using default value of 1 
joint mimic: no offset, using default value of 0 
URDFParser: Link size: 17
URDFParser: Joint size: 12
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jrl/urdfs/panda/meshes/visual/link0.dae (59388 verts, 20478 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jrl/urdfs/panda/meshes/visual/link1.dae (37309 verts, 12516 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jrl/urdfs/panda/meshes/visual/link2.dae (37892 verts, 12716 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jrl/urdfs/panda/meshes/visual/link3.dae (42512 verts, 14233 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jrl/urdfs/panda/meshes/visual/link4.dae (43520 verts, 14620 tris)
LoadAssimp: Loaded model /ho

In [9]:
solver.path_following(load_time='1111215818', num_traj=100, shrink_ratio=0, enable_evaluation=True, enable_plot=False)

using shrink_ratio: 0
/home/luca/ikpflow/data/panda/trajectory/1111215818/ load successfully.
move C to cuda
======135=(0.0617515717053927)======
move C to cuda
======327=(0.1324026865030461)======
move C to cuda
======264=(0.06187335900416387)======
move C to cuda
======584=(0.07589537518876238)======
move C to cuda
======23=(-0.04520876308650289)======
move C to cuda
======59=(-0.04203762810408255)======
move C to cuda
======71=(0.13488641403218546)======
move C to cuda
======113=(0.13457478735932396)======
move C to cuda
======562=(0.13266472901173088)======
move C to cuda
======370=(0.0734730852363279)======
move C to cuda
======491=(0.06546351324710419)======
move C to cuda
======65=(-0.04787858802773372)======
move C to cuda
======87=(-0.04460016884339407)======
move C to cuda
======379=(-0.14206593055871133)======
move C to cuda
======584=(0.07589537518876238)======
move C to cuda
======498=(-0.047870448959501634)======
move C to cuda
======321=(0.06160162011197638)======
move C

In [27]:
import numpy as np
import pandas as pd
from time import time
from ikflow.utils import set_seed
from ikflow.model_loading import get_ik_solver

In [5]:
set_seed()

# Build IKFlowSolver and set weights
ik_solver, hyper_parameters = get_ik_solver("panda__full__lp191_5.25m")
robot = ik_solver.robot

P = solver._sample_P_path(load_time='1111215818')

set_seed() - random int:  541
WorldModel::LoadRobot: /tmp/panda_arm_hand_formatted_link_filepaths_absolute.urdf
joint mimic: no multiplier, using default value of 1 
joint mimic: no offset, using default value of 0 
URDFParser: Link size: 17
URDFParser: Joint size: 12
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jkinpylib/urdfs/panda/meshes/visual/link0.dae (59388 verts, 20478 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jkinpylib/urdfs/panda/meshes/visual/link1.dae (37309 verts, 12516 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jkinpylib/urdfs/panda/meshes/visual/link2.dae (37892 verts, 12716 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jkinpylib/urdfs/panda/meshes/visual/link3.dae (42512 verts, 14233 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jkinpylib/urdfs/panda/meshes/visual/link4.dae (43520 verts, 14620 tr

In [16]:
num_traj = 100
num_steps = P.shape[0]

Qs = np.empty((num_traj, num_steps, solver._robot.n_dofs))
l2_err_arr = np.empty((num_traj, num_steps))
ang_err_arr = np.empty((num_traj, num_steps))
mjac_arr = np.empty((num_traj, num_steps))
runtime_arr = np.empty((num_traj))

In [28]:
begin = time()
for i in range(num_traj):
    sol, l2_err_arr[i], ang_err_arr[i], _, _, runtime = ik_solver.solve_n_poses(
        P, 
        latent_scale=.01,
        refine_solutions=False, 
        return_detailed=True)
    # runtime_arr[i] = runtime
    mjac_arr[i] = solver.max_joint_angle_change(sol)
    Qs[i] = sol.detach().cpu().numpy()

avg_runtime = (time() - begin) / num_traj

In [29]:
df = pd.DataFrame({'l2_err': l2_err_arr.flatten(), 'ang_err': ang_err_arr.flatten(), 'mjac': mjac_arr.flatten()})
print(df.describe())
print(f"avg_inference_time: {avg_runtime}")

            l2_err      ang_err         mjac
count  2000.000000  2000.000000  2000.000000
mean      0.044247     0.329984   271.592331
std       0.026703     0.205454    18.936939
min       0.002211     0.002674   208.083664
25%       0.019647     0.184259   270.516106
50%       0.047693     0.331568   279.244812
75%       0.060714     0.479523   281.726219
max       0.200551     2.576473   306.408539
avg_inference_time: 0.024351565837860106
